In [1]:
import os

CAR_PATH = os.path.join("~\my_env\datasets\cars", "car.txt")

In [2]:
import pandas as pd 
cars = pd.read_csv(CAR_PATH, delimiter = ",", header=None)
cars.columns = ["Buying", "Maint", "Doors", "Persons", "Lug_boot", "Safety", "Acceptability"]
cars = cars.reset_index()
carsArray = cars.to_numpy()
cars

,index,Buying,Maint,Doors,Persons,Lug_boot,Safety,Acceptability
0,0,vhigh,vhigh,2,2,small,low,unacc
1,1,vhigh,vhigh,2,2,small,med,unacc
2,2,vhigh,vhigh,2,2,small,high,unacc
3,3,vhigh,vhigh,2,2,med,low,unacc
4,4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...,...
1723,1723,low,low,5more,more,med,med,good
1724,1724,low,low,5more,more,med,high,vgood
1725,1725,low,low,5more,more,big,low,unacc
1726,1726,low,low,5more,more,big,med,good


In [3]:
for col in cars.columns:
    print(cars[col].value_counts())

1727    1
1158    1
1136    1
1138    1
1140    1
       ..
583     1
585     1
587     1
589     1
0       1
Name: index, Length: 1728, dtype: int64
high     432
med      432
low      432
vhigh    432
Name: Buying, dtype: int64
high     432
med      432
low      432
vhigh    432
Name: Maint, dtype: int64
3        432
4        432
5more    432
2        432
Name: Doors, dtype: int64
more    576
4       576
2       576
Name: Persons, dtype: int64
big      576
small    576
med      576
Name: Lug_boot, dtype: int64
high    576
med     576
low     576
Name: Safety, dtype: int64
unacc    1210
acc       384
good       69
vgood      65
Name: Acceptability, dtype: int64


In [4]:
# We have to stratify Acceptability since it's skewed
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(cars, cars[["Acceptability"]]): # Using StratifiedShuffleSplit and splitting using housing["income_cat"], we ensured that the proportion of each category of housing["income_cat"] is the same in the training and test sets
    strat_train_full_set = cars.loc[train_index]
    strat_test_set = cars.loc[test_index]
    
for train_index, valid_index in split.split(strat_train_full_set, strat_train_full_set[["Acceptability"]]):
    strat_train_set = strat_train_full_set.iloc[train_index] # loc does not work
    strat_valid_set = strat_train_full_set.iloc[valid_index]

In [5]:
strat_train_set["Acceptability"].value_counts() / len(strat_train_set)

unacc    0.700452
acc      0.221719
good     0.039819
vgood    0.038009
Name: Acceptability, dtype: float64

In [6]:
strat_test_set["Acceptability"].value_counts() / len(strat_test_set)

unacc    0.699422
acc      0.222543
good     0.040462
vgood    0.037572
Name: Acceptability, dtype: float64

In [7]:
strat_train_set

,index,Buying,Maint,Doors,Persons,Lug_boot,Safety,Acceptability
20,20,vhigh,vhigh,2,more,small,high,unacc
1462,1462,low,high,4,2,med,med,unacc
691,691,high,med,3,4,big,med,acc
288,288,vhigh,med,4,more,small,low,unacc
1603,1603,low,med,5more,4,small,med,acc
...,...,...,...,...,...,...,...,...
1013,1013,med,high,3,4,med,high,acc
24,24,vhigh,vhigh,2,more,big,low,unacc
293,293,vhigh,med,4,more,med,high,acc
241,241,vhigh,med,2,more,big,med,acc


In [8]:
strat_train_set_data = strat_train_set.drop("Acceptability", axis=1) # drop labels for training set
strat_train_set_labels = strat_train_set["Acceptability"].copy()
strat_valid_set_data = strat_valid_set.drop("Acceptability", axis=1) # drop labels for training set
strat_valid_set_labels = strat_valid_set["Acceptability"].copy()

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler


data_cat_attribs = ["Buying", "Maint", "Doors", "Persons", "Lug_boot", "Safety"]

data_pipeline = ColumnTransformer([
        ("oHCat", OneHotEncoder(), data_cat_attribs),
])

strat_train_set_data_encoded = data_pipeline.fit_transform(strat_train_set_data)
strat_valid_set_data_encoded = data_pipeline.fit_transform(strat_valid_set_data)

In [10]:
strat_train_set_data_encoded

<1105x21 sparse matrix of type '<class 'numpy.float64'>'
	with 6630 stored elements in Compressed Sparse Row format>

In [11]:
import numpy as np
strat_train_set_labels = np.array(strat_train_set_labels.values.tolist())
strat_train_set_labels = strat_train_set_labels.reshape(-1, 1) 
strat_valid_set_labels = np.array(strat_valid_set_labels.values.tolist())
strat_valid_set_labels = strat_valid_set_labels.reshape(-1, 1)

In [12]:
ordinal_encoder = OrdinalEncoder()
strat_train_set_labels_encoded = ordinal_encoder.fit_transform(strat_train_set_labels) # Change different text categories to numbers
strat_valid_set_labels_encoded = ordinal_encoder.fit_transform(strat_valid_set_labels)
strat_train_set_labels_encoded[:10]

array([[2.],
       [2.],
       [0.],
       [2.],
       [0.],
       [2.],
       [0.],
       [2.],
       [2.],
       [0.]])

In [13]:
ordinal_encoder.categories_

[array(['acc', 'good', 'unacc', 'vgood'], dtype='<U5')]

In [39]:
strat_train_set_labels_encoded = strat_train_set_labels_encoded.flatten()
strat_valid_set_labels_encoded = strat_valid_set_labels_encoded.flatten()

In [15]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter = 5000, random_state=42)
sgd_clf.fit(strat_train_set_data_encoded, strat_train_set_labels_encoded)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=5000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [16]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

skfolds = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)
clone_clf = clone(sgd_clf)
for train_index, test_index in skfolds.split(strat_train_set_data_encoded, strat_train_set_labels_encoded): # provide train and test index such that in each fold-validation, the class 5 & class not-5 proportions of the training and test sets are the same as the entire dataset.
    X_train_folds = strat_train_set_data_encoded[train_index]
    y_train_folds = strat_train_set_labels_encoded[train_index]
    X_test_fold = strat_train_set_data_encoded[test_index]
    y_test_fold = strat_train_set_labels_encoded[test_index]
    
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))

0.8590785907859079
0.8590785907859079
0.8555858310626703


In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [10,30,100], 'max_features': [8,10,12]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [10,30,100], 'max_features': [8,10,12]},
  ]

forest_clf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(forest_clf, param_grid, cv=5,
                           scoring='f1_weighted',
                           return_train_score=True)
grid_search.fit(strat_train_set_data_encoded, strat_train_set_labels_encoded)



for_clf = grid_search.best_estimator_
skfolds = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)
clone_clf = clone(for_clf)
for train_index, test_index in skfolds.split(strat_train_set_data_encoded, strat_train_set_labels_encoded): # provide train and test index such that in each fold-validation, the class 5 & class not-5 proportions of the training and test sets are the same as the entire dataset.
    X_train_folds = strat_train_set_data_encoded[train_index]
    y_train_folds = strat_train_set_labels_encoded[train_index]
    X_test_fold = strat_train_set_data_encoded[test_index]
    y_test_fold = strat_train_set_labels_encoded[test_index]
    
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
#     y_pred = y_pred.reshape(-1,1)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))

0.943089430894309
0.9349593495934959
0.9509536784741145


In [18]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.929691182772363 {'max_features': 8, 'n_estimators': 10}
0.9403809751274504 {'max_features': 8, 'n_estimators': 30}
0.9500815521965411 {'max_features': 8, 'n_estimators': 100}
0.9293743861022711 {'max_features': 10, 'n_estimators': 10}
0.9457516845255892 {'max_features': 10, 'n_estimators': 30}
0.9485008875115397 {'max_features': 10, 'n_estimators': 100}
0.9455154048776516 {'max_features': 12, 'n_estimators': 10}
0.9567003357810132 {'max_features': 12, 'n_estimators': 30}
0.9557733315721635 {'max_features': 12, 'n_estimators': 100}
0.9428576453627583 {'bootstrap': False, 'max_features': 8, 'n_estimators': 10}
0.9498782113510762 {'bootstrap': False, 'max_features': 8, 'n_estimators': 30}
0.9475272112434501 {'bootstrap': False, 'max_features': 8, 'n_estimators': 100}
0.945088272530604 {'bootstrap': False, 'max_features': 10, 'n_estimators': 10}
0.9512309058215088 {'bootstrap': False, 'max_features': 10, 'n_estimators': 30}
0.9557371218490074 {'bootstrap': False, 'max_features': 10, 'n_e

In [19]:
grid_search.best_params_

{'max_features': 12, 'n_estimators': 30}

In [20]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(strat_train_set_data_encoded, strat_train_set_labels_encoded)

skfolds = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)
clone_clf = clone(log_reg)
for train_index, test_index in skfolds.split(strat_train_set_data_encoded, strat_train_set_labels_encoded): # provide train and test index such that in each fold-validation, the class 5 & class not-5 proportions of the training and test sets are the same as the entire dataset.
    X_train_folds = strat_train_set_data_encoded[train_index]
    y_train_folds = strat_train_set_labels_encoded[train_index]
    X_test_fold = strat_train_set_data_encoded[test_index]
    y_test_fold = strat_train_set_labels_encoded[test_index]
    
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
#     y_pred = y_pred.reshape(-1,1)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))

0.8753387533875339
0.8644986449864499
0.8664850136239782


In [21]:
from sklearn.model_selection import cross_val_predict

y_valid_pred = cross_val_predict(for_clf, strat_valid_set_data_encoded, strat_valid_set_labels_encoded, cv=3)

In [22]:
from sklearn.metrics import confusion_matrix
confusion_matrix(strat_valid_set_labels_encoded, y_valid_pred) # arguments: actual, predict

array([[ 48,   7,   7,   0],
       [  7,   4,   0,   0],
       [ 10,   0, 184,   0],
       [  5,   0,   0,   5]], dtype=int64)

In [23]:
from sklearn.metrics import precision_score, recall_score
precision_score(strat_valid_set_labels_encoded, y_valid_pred, average='weighted')

0.8787160219144645

In [24]:
recall_score(strat_valid_set_labels_encoded, y_valid_pred, average='weighted')

0.8700361010830325

In [25]:
from sklearn.metrics import f1_score

f1_score(strat_valid_set_labels_encoded, y_valid_pred, average='weighted') # the greater the precison & recall, the greater the f1_score and the better the classifier!

0.8707268663947365

In [26]:
from sklearn.metrics import classification_report

y_pred = for_clf.predict(strat_valid_set_data_encoded)
print(classification_report(strat_valid_set_labels_encoded, y_pred))

              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97        62
         1.0       1.00      0.82      0.90        11
         2.0       0.99      0.99      0.99       194
         3.0       0.83      1.00      0.91        10

    accuracy                           0.98       277
   macro avg       0.95      0.95      0.94       277
weighted avg       0.98      0.98      0.98       277



In [27]:
import tensorflow as tf
from tensorflow import keras
def build_model(n_hidden=3, n_neurons=30, learning_rate=3e-3, input_shape=[21]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(4, activation="softmax")) # activation="sigmoid" for binary outputs
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(metrics= ['acc'], loss="sparse_categorical_crossentropy", optimizer=optimizer)
    return model

In [28]:
keras_reg = keras.wrappers.scikit_learn.KerasClassifier(build_model)

In [29]:
type(strat_valid_set_data_encoded)

scipy.sparse.csr.csr_matrix

In [30]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2),
}

strat_train_set_data_encoded = strat_train_set_data_encoded.toarray()
# strat_train_set_labels_encoded = strat_train_set_labels_encoded.toarray()
strat_valid_set_data_encoded = strat_valid_set_data_encoded.toarray()
# strat_valid_set_labels_encoded = strat_valid_set_labels_encoded.toarray()

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(strat_train_set_data_encoded, strat_train_set_labels_encoded, epochs=100,
                  validation_data=(strat_valid_set_data_encoded, strat_valid_set_labels_encoded),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] learning_rate=0.027964175963566883, n_hidden=3, n_neurons=4 .....


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Train on 736 samples, validate on 277 samples
Epoch 1/100
736/736 [==============================] - 1s 946us/sample - loss: 1.0982 - acc: 0.7052 - val_loss: 0.9348 - val_acc: 0.7004
Epoch 2/100
736/736 [==============================] - 0s 105us/sample - loss: 0.8501 - acc: 0.7038 - val_loss: 0.8084 - val_acc: 0.7004
Epoch 3/100
736/736 [==============================] - 0s 122us/sample - loss: 0.7689 - acc: 0.7038 - val_loss: 0.7635 - val_acc: 0.7004
Epoch 4/100
736/736 [==============================] - 0s 99us/sample - loss: 0.7278 - acc: 0.7038 - val_loss: 0.7271 - val_acc: 0.7004
Epoch 5/100
736/736 [==============================] - 0s 109us/sample - loss: 0.6919 - acc: 0.7038 - val_loss: 0.6949 - val_acc: 0.7004
Epoch 6/100
736/736 [==============================] - 0s 88us/sample - loss: 0.6573 - acc: 0.7038 - val_loss: 0.6624 - val_acc: 0.7004
Epoch 7/100
736/736 [==============================] - 0s 99us/sample - loss: 0.6221 - acc: 0.7038 - val_loss: 0.6290 - val_acc: 0.700

736/736 [==============================] - 0s 105us/sample - loss: 0.2420 - acc: 0.8859 - val_loss: 0.2504 - val_acc: 0.8736
Epoch 61/100
736/736 [==============================] - 0s 101us/sample - loss: 0.2410 - acc: 0.8832 - val_loss: 0.2561 - val_acc: 0.8736
Epoch 62/100
736/736 [==============================] - 0s 94us/sample - loss: 0.2416 - acc: 0.8832 - val_loss: 0.2523 - val_acc: 0.8736
Epoch 63/100
736/736 [==============================] - 0s 88us/sample - loss: 0.2372 - acc: 0.8913 - val_loss: 0.2570 - val_acc: 0.8700
Epoch 64/100
736/736 [==============================] - 0s 92us/sample - loss: 0.2378 - acc: 0.8954 - val_loss: 0.2580 - val_acc: 0.8736
Epoch 65/100
736/736 [==============================] - 0s 88us/sample - loss: 0.2355 - acc: 0.8927 - val_loss: 0.2542 - val_acc: 0.8809
Epoch 66/100
736/736 [==============================] - 0s 88us/sample - loss: 0.2343 - acc: 0.8954 - val_loss: 0.2411 - val_acc: 0.8953
Epoch 67/100
736/736 [==============================

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.1s remaining:    0.0s


...

Epoch 90/100
737/737 [==============================] - 0s 107us/sample - loss: 0.6805 - acc: 0.7164 - val_loss: 0.6752 - val_acc: 0.7329
Epoch 91/100
737/737 [==============================] - 0s 107us/sample - loss: 0.6787 - acc: 0.7164 - val_loss: 0.6737 - val_acc: 0.7329
Epoch 92/100
737/737 [==============================] - 0s 103us/sample - loss: 0.6769 - acc: 0.7164 - val_loss: 0.6722 - val_acc: 0.7329
Epoch 93/100
737/737 [==============================] - 0s 95us/sample - loss: 0.6752 - acc: 0.7178 - val_loss: 0.6707 - val_acc: 0.7329
Epoch 94/100
737/737 [==============================] - 0s 91us/sample - loss: 0.6735 - acc: 0.7178 - val_loss: 0.6692 - val_acc: 0.7329
Epoch 95/100
737/737 [==============================] - 0s 90us/sample - loss: 0.6718 - acc: 0.7178 - val_loss: 0.6675 - val_acc: 0.7329
Epoch 96/100
737/737 [==============================] - 0s 87us/sample - loss: 0.6699 - acc: 0.7191 - val_loss: 0.6660 - val_acc: 0.7329
Epoch 97/100
737/737 [================

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  4.1min finished


1105/1105 [==============================] - 1s 492us/sample - loss: 1.0607 - acc: 0.6833 - val_loss: 0.9136 - val_acc: 0.7004
Epoch 2/100
1105/1105 [==============================] - 0s 96us/sample - loss: 0.8570 - acc: 0.7005 - val_loss: 0.8175 - val_acc: 0.7004
Epoch 3/100
1105/1105 [==============================] - 0s 91us/sample - loss: 0.7812 - acc: 0.7005 - val_loss: 0.7590 - val_acc: 0.7004
Epoch 4/100
1105/1105 [==============================] - 0s 89us/sample - loss: 0.7257 - acc: 0.7005 - val_loss: 0.7107 - val_acc: 0.7004
Epoch 5/100
1105/1105 [==============================] - 0s 91us/sample - loss: 0.6753 - acc: 0.7041 - val_loss: 0.6650 - val_acc: 0.7184
Epoch 6/100
1105/1105 [==============================] - 0s 89us/sample - loss: 0.6272 - acc: 0.7294 - val_loss: 0.6212 - val_acc: 0.7473
Epoch 7/100
1105/1105 [==============================] - 0s 92us/sample - loss: 0.5826 - acc: 0.7557 - val_loss: 0.5805 - val_acc: 0.7690
Epoch 8/100
1105/1105 [======================

1105/1105 [==============================] - 0s 94us/sample - loss: 0.1210 - acc: 0.9665 - val_loss: 0.1481 - val_acc: 0.9567
Epoch 61/100
1105/1105 [==============================] - 0s 98us/sample - loss: 0.1178 - acc: 0.9701 - val_loss: 0.1380 - val_acc: 0.9458
Epoch 62/100
1105/1105 [==============================] - 0s 103us/sample - loss: 0.1175 - acc: 0.9683 - val_loss: 0.1349 - val_acc: 0.9531
Epoch 63/100
1105/1105 [==============================] - 0s 103us/sample - loss: 0.1143 - acc: 0.9701 - val_loss: 0.1365 - val_acc: 0.9603
Epoch 64/100
1105/1105 [==============================] - 0s 97us/sample - loss: 0.1116 - acc: 0.9683 - val_loss: 0.1407 - val_acc: 0.9639
Epoch 65/100
1105/1105 [==============================] - 0s 91us/sample - loss: 0.1100 - acc: 0.9738 - val_loss: 0.1334 - val_acc: 0.9639
Epoch 66/100
1105/1105 [==============================] - 0s 93us/sample - loss: 0.1082 - acc: 0.9738 - val_loss: 0.1293 - val_acc: 0.9639
Epoch 67/100
1105/1105 [==============

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x000000174C2C4BC8>,
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000017514F1288>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,...
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score

In [31]:
rnd_search_cv.best_params_

{'learning_rate': 0.022438738977004893, 'n_hidden': 2, 'n_neurons': 43}

In [32]:
rnd_search_cv.best_score_ # score is the higher (more positive) the better

0.922171932108262

In [33]:
cvres = rnd_search_cv.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]): # mean score is the higher (more positive) the better, since this is sklearn's RandomizedSearchCV
    print(mean_score, params)

0.8235294145696304 {'learning_rate': 0.027964175963566883, 'n_hidden': 3, 'n_neurons': 4}
0.8561086118490987 {'learning_rate': 0.004268832994870721, 'n_hidden': 2, 'n_neurons': 56}
0.8506787519109734 {'learning_rate': 0.007841984339361164, 'n_hidden': 1, 'n_neurons': 54}
0.780995470312386 {'learning_rate': 0.0042842284194449624, 'n_hidden': 0, 'n_neurons': 5}
0.7312217062415041 {'learning_rate': 0.0015612574596896014, 'n_hidden': 1, 'n_neurons': 37}
0.7076923278661874 {'learning_rate': 0.003440557417533077, 'n_hidden': 3, 'n_neurons': 8}
0.922171932108262 {'learning_rate': 0.022438738977004893, 'n_hidden': 2, 'n_neurons': 43}
0.8678732961551097 {'learning_rate': 0.006565007854155186, 'n_hidden': 2, 'n_neurons': 89}
0.7131221650952128 {'learning_rate': 0.002181278902927947, 'n_hidden': 0, 'n_neurons': 91}
0.7257918532078083 {'learning_rate': 0.002088882136889992, 'n_hidden': 0, 'n_neurons': 67}


In [34]:
rnd_search_cv.best_estimator_

In [35]:
model = rnd_search_cv.best_estimator_.model
model

In [36]:
ANNmodel = model

In [37]:
from sklearn.metrics import classification_report

y_pred = ANNmodel.predict_classes(strat_valid_set_data_encoded)
print(classification_report(strat_valid_set_labels_encoded, y_pred))

              precision    recall  f1-score   support

         0.0       0.95      0.92      0.93        62
         1.0       1.00      0.73      0.84        11
         2.0       0.97      0.99      0.98       194
         3.0       0.91      1.00      0.95        10

    accuracy                           0.97       277
   macro avg       0.96      0.91      0.93       277
weighted avg       0.97      0.97      0.97       277



In [40]:
for_clf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features=12, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [41]:
strat_train_full_set_data = strat_train_full_set.drop("Acceptability", axis=1)
strat_train_full_set_labels = strat_train_full_set["Acceptability"].copy()
strat_train_full_set_data_encoded = data_pipeline.fit_transform(strat_train_full_set_data)
strat_train_full_set_labels = np.array(strat_train_full_set_labels.values.tolist())
strat_train_full_set_labels = strat_train_full_set_labels.reshape(-1, 1)
strat_train_full_set_labels_encoded = ordinal_encoder.fit_transform(strat_train_full_set_labels)
strat_train_full_set_labels_encoded = strat_train_full_set_labels_encoded.flatten()

# Use full_data (including train and valid to train)
for_clf.fit(strat_train_full_set_data_encoded,strat_train_full_set_labels_encoded)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features=12, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [42]:
from sklearn.metrics import mean_squared_error

final_model = for_clf
X_test = strat_test_set.drop("Acceptability", axis=1)
y_test = strat_test_set["Acceptability"].copy()
y_test = np.array(y_test.values.tolist())
y_test = y_test.reshape(-1, 1) 

X_test_prepared = data_pipeline.transform(X_test) # we do not fit here because we want to use the fitted values of the training set to predict this, i.e., use the training set's median values and extra attributes to transform the test set
y_test = ordinal_encoder.fit_transform(y_test)

final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [43]:
final_rmse

0.2741249873151301

In [44]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, final_predictions)

array([[ 76,   0,   0,   1],
       [  0,  14,   0,   0],
       [  2,   0, 240,   0],
       [  1,   0,   0,  12]], dtype=int64)

In [45]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_test, final_predictions, average='weighted')

0.988658813199678

In [46]:
recall_score(y_test, final_predictions, average='weighted')

0.9884393063583815

In [47]:
from sklearn.metrics import f1_score

f1_score(y_test, final_predictions, average='weighted') 

0.9885014209481147

In [52]:
y_pred = for_clf.predict(X_test_prepared)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.96      0.99      0.97        77
         1.0       1.00      1.00      1.00        14
         2.0       1.00      0.99      1.00       242
         3.0       0.92      0.92      0.92        13

    accuracy                           0.99       346
   macro avg       0.97      0.98      0.97       346
weighted avg       0.99      0.99      0.99       346

